In [1]:
# Import packages
import os
import glob
import shutil
import plotnine

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

from plotnine import ggplot
from plotnine import aes 
from plotnine import geom_line
from plotnine import labs

In [2]:
# Reproducibility
np.random.seed(478)
tf.random.set_seed(478)

In [3]:
# Read in data from .csv files
train = pd.read_csv('./Data/train.csv')
test = pd.read_csv('./Data/test.csv')

# Convert Pandas dataframes to valid Numpy arrays
def convert(X):
    data = np.empty((X.shape[0], 28, 28, 1))
    for i in range(X.shape[0]):
        img = np.array(X.iloc[i, :]).reshape((28, 28, 1))
        data[i] = img
    return(np.array(data))

# Define training set
X_train = convert(train.drop('label', axis = 1)) / 255
y_train = train['label']

# Define testing set
X_test = convert(test) / 255

In [9]:
model = keras.Sequential([
    layers.BatchNormalization(),
    layers.DepthwiseConv2D(kernel_size = 3, activation = 'relu', depth_multiplier = 4, input_shape = [28, 28, 1]),
    layers.MaxPool2D(),
    layers.Dropout(rate = 0.3),
    
    layers.BatchNormalization(),
    layers.DepthwiseConv2D(kernel_size = 3, activation = 'relu', depth_multiplier = 4),
    layers.MaxPool2D(),
    layers.Dropout(rate = 0.3),
    
    layers.Flatten(),
    layers.Dense(units = 10, activation = tf.nn.softmax)
])

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# Compile model
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

# Train model with 75/25 training/validation split
trained = model.fit(
    X_train,
    y_train,
    epochs = 20,
    validation_split = .25,
    callbacks = [early_stopping]
)

Epoch 1/20
985/985 [==============================] - 49s 49ms/step - loss: 0.5980 - accuracy: 0.8048 - val_loss: 0.1571 - val_accuracy: 0.9570
Epoch 2/20
985/985 [==============================] - 47s 48ms/step - loss: 0.2366 - accuracy: 0.9255 - val_loss: 0.1154 - val_accuracy: 0.9683
Epoch 3/20
985/985 [==============================] - 48s 49ms/step - loss: 0.2006 - accuracy: 0.9372 - val_loss: 0.1000 - val_accuracy: 0.9710
Epoch 4/20
985/985 [==============================] - 46s 47ms/step - loss: 0.1798 - accuracy: 0.9441 - val_loss: 0.1004 - val_accuracy: 0.9713
Epoch 5/20
985/985 [==============================] - 47s 48ms/step - loss: 0.1665 - accuracy: 0.9473 - val_loss: 0.0854 - val_accuracy: 0.9761
Epoch 6/20
985/985 [==============================] - 47s 48ms/step - loss: 0.1641 - accuracy: 0.9471 - val_loss: 0.0895 - val_accuracy: 0.9761
Epoch 7/20
985/985 [==============================] - 47s 48ms/step - loss: 0.1599 - accuracy: 0.9493 - val_loss: 0.0846 - val_accuracy:

In [11]:
# Generate competition preedictions based on test set
pred = pd.DataFrame(model.predict(X_test)).idxmax(axis = 1)